<a href="https://colab.research.google.com/github/pentai23/BitkubAndBinance/blob/main/BIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import hashlib
import hmac
import json
import requests

# API info
API_HOST = 'https://api.bitkub.com'
API_KEY = '7a047a9efb2b3db40d5c385ff9556bd6'
API_SECRET = b'03bd0111f5f9583f243d78c01e8740a1'

def json_encode(data):
	return json.dumps(data, separators=(',', ':'), sort_keys=True)

def sign(data):
	j = json_encode(data)
	print('Signing payload: ' + j)
	h = hmac.new(API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
	return h.hexdigest()

# check server time
response = requests.get(API_HOST + '/api/servertime')
ts = int(response.text)
print('Server time: ' + response.text)

# check balances
header = {
	'Accept': 'application/json',
	'Content-Type': 'application/json',
	'X-BTK-APIKEY': API_KEY,
}
data = {
	'ts': ts,
}
signature = sign(data)
data['sig'] = signature

print('Payload with signature: ' + json_encode(data))
response = requests.post(API_HOST + '/api/market/balances', headers=header, data=json_encode(data))

print('Balances: ' + response.text)

Server time: 1646842622
Signing payload: {"ts":1646842622}
Payload with signature: {"sig":"b4b9595d81ff5d0409705525ccc7bcb2f9fb14c816cf276148d0a17cb7381d27","ts":1646842622}
Balances: {"error":0,"result":{"BTC":{"available":0,"reserved":0},"ETH":{"available":0,"reserved":0},"WAN":{"available":0,"reserved":0},"ADA":{"available":0,"reserved":0},"OMG":{"available":0,"reserved":0},"BCH":{"available":0,"reserved":0},"USDT":{"available":0,"reserved":0},"LTC":{"available":0,"reserved":0},"XRP":{"available":0,"reserved":0},"ZIL":{"available":0,"reserved":0},"SNT":{"available":0,"reserved":0},"CVC":{"available":0,"reserved":0},"LINK":{"available":0,"reserved":0},"IOST":{"available":0,"reserved":0},"ZRX":{"available":0,"reserved":0},"KNC":{"available":0,"reserved":0},"ABT":{"available":0,"reserved":0},"MANA":{"available":0,"reserved":0},"CTXC":{"available":0,"reserved":0},"XLM":{"available":0,"reserved":0},"SIX":{"available":0,"reserved":0},"JFIN":{"available":0,"reserved":0},"BNB":{"available":

In [17]:
# bitkubchecker.py

import requests
from pprint import pprint
import time
from songline import Sendline
# pip install songline
token = 'bgClpKkgX4c4o0k0pAMLmd7pfSgSotcGpHAF0tT8N43'
messenger = Sendline(token)

line_condition = True

API_HOST = 'https://api.bitkub.com'
mycoin = ['THB_BTC',
          'THB_ETH',
          'THB_DOGE',
          'THB_ETH',
          'THB_LTC',
          'THB_USDT',
          'THB_XRP',
          'THB_KSM']


condition = {'THB_BTC':{'buy':950000,'sell':1005000},
             'THB_ETH':{'buy':20000,'sell':30000},
             'THB_DOGE':{'buy':0.389,'sell':0.4}}


#print(condition['THB_BTC'])

def CheckCondition(coin,price):
    # coin= 'THB_BTC', price = 1050000
    text = ''
    check_buy = condition[coin]['buy']
    if price <= check_buy:
        txt = '{} ราคาลงแล้ว เหลือ: {:,.3f} รีบซื้อด่วน!\n(ราคาที่อยากได้: {:,.3f})'.format(coin,price,check_buy)
        #print(txt)
        text += txt + '\n'

    check_sell = condition[coin]['sell']
    if price >= check_sell:
        txt = '{} ราคาขึ้นแล้ว ล่าสุดเป็น: {:,.3f} รีบขายด่วน!\n(ราคาที่อยากขาย: {:,.3f})'.format(coin,price,check_sell)
        #print(txt)
        text += txt + '\n'

    return text



current_text = ''


def CheckPrice():
    global current_text
    response = requests.get(API_HOST + '/api/market/ticker')
    result = response.json()

    #pprint(result['THB_BTC'])

    alltext = ''
    text_line = ''
    
    for c in mycoin:
        #sym = 'THB_BTC'
        sym = c
        data = result[sym]
        last = data['last']
        #print(data)
        #print(sym, last)
        pchange = data['percentChange']
        text = '{} price: {:,.3f} ({})'.format(sym,last,pchange)
        alltext += text + '\n' # alltext = alltext + text
        if line_condition == True:
            if c in condition:
                checktext = CheckCondition(c,last)
                if len(checktext) > 0:
                    text_line += checktext
                    
    # เงื่อนไขที่จะส่งไลน์ (ดูคลิปเลย)               
    if line_condition == True and current_text != text_line:
        print('Condition: ',text_line)
        current_text = text_line
        # sendline
        messenger.sendtext(text_line)
        
        
        
        
        
    #print(type(last))
    v_result.set(alltext)
    #print('-----')
    R1.after(100,CheckPrice)
    # .after คือ การทำให้ R1 มีการ refresh ทุก 200 ms เรียกฟังชั่น CheckPrice
    


######################GUI######################

from tkinter import *
from tkinter import ttk

GUI = Tk()
GUI.geometry('500x600')
GUI.title('โปรแกรมเช็คราคาจาก Bitkub')

FONT1 = ('Angsana New',30)

L1 = ttk.Label(GUI,text='ราคา bitkub ล่าสุด',font=FONT1)
L1.pack()

#B1 = ttk.Button(GUI,text='Check!',command=CheckPrice)
#B1.pack(ipadx=20,ipady=10)

v_result = StringVar()
v_result.set('--------result--------')
R1 = ttk.Label(textvariable=v_result,font=FONT1)
R1.pack()

#Run ฟังชั่นทุกครั้งที่เปิด
CheckPrice()
GUI.mainloop()


ModuleNotFoundError: ignored

In [ ]:
apikey = '7a047a9efb2b3db40d5c385ff9556bd6' 
apisecret = b'03bd0111f5f9583f243d78c01e8740a1'

#API KEY , API SECRET นำมาจากลิ้งค์นี้ https://www.bitkub.com/publicapi
# ห้ามลบตัว b ออกหน้ารหัส ของ apisecret

import hashlib
import hmac
import json
import requests

# API info
API_HOST = 'https://api.bitkub.com'
API_KEY = apikey
API_SECRET = apisecret

def json_encode(data):
	return json.dumps(data, separators=(',', ':'), sort_keys=True)

def sign(data):
	j = json_encode(data)
	print('Signing payload: ' + j)
	h = hmac.new(API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
	return h.hexdigest()

# check server time
response = requests.get(API_HOST + '/api/servertime')
ts = int(response.text)
print('Server time: ' + response.text)

# place bid
header = {
	'Accept': 'application/json',
	'Content-Type': 'application/json',
	'X-BTK-APIKEY': API_KEY,
}
data = {
	'sym': 'THB_DOGE',
	'amt': 200, # THB amount you want to spend
	'rat': 0.29,
	'typ': 'limit',
	'ts': ts,
}
signature = sign(data)
data['sig'] = signature

print('Payload with signature: ' + json_encode(data))
response = requests.post(API_HOST + '/api/market/place-bid', headers=header, data=json_encode(data))

print('Response: ' + response.text)

Server time: 1646839357
Signing payload: {"amt":200,"rat":0.29,"sym":"THB_DOGE","ts":1646839357,"typ":"limit"}
Payload with signature: {"amt":200,"rat":0.29,"sig":"924f32364e46becfe98db15d0fee2317f197df2c7d8a6ad0f26db7a57866d803","sym":"THB_DOGE","ts":1646839357,"typ":"limit"}
Response: {"error":18}


In [3]:
# bitkubchecker.py

import requests
from pprint import pprint
import time

API_HOST = 'https://api.bitkub.com'

mycoin = ['THB_BTC','THB_ETH','THB_DOGE']

while True:
    response = requests.get(API_HOST + '/api/market/ticker')
    result = response.json()

    #pprint(result['THB_BTC'])
    for c in mycoin:
        #sym = 'THB_BTC'
        sym = c
        data = result[sym]
        last = data['last']
        #print(data)
        print(sym, last)
    print('-----')
    time.sleep(0.2)


THB_BTC 1400000
THB_ETH 91208.45
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.45
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.45
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.45
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.45
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 91208.51
THB_DOGE 4.071
-----
THB_BTC 1400000
THB_ETH 9120

KeyboardInterrupt: ignored

In [10]:
apikey = '7a047a9efb2b3db40d5c385ff9556bd6' 
apisecret = b'03bd0111f5f9583f243d78c01e8740a1'

#API KEY , API SECRET นำมาจากลิ้งค์นี้ https://www.bitkub.com/publicapi
# ห้ามลบตัว b ออกหน้ารหัส ของ apisecret

import hashlib
import hmac
import json
import requests


# API info
API_HOST = 'https://api.bitkub.com'
API_KEY = apikey
API_SECRET = apisecret

def json_encode(data):
	return json.dumps(data, separators=(',', ':'), sort_keys=True)

def sign(data):
	j = json_encode(data)
	print('Signing payload: ' + j)
	h = hmac.new(API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
	return h.hexdigest()

# check server time
response = requests.get(API_HOST + '/api/servertime')
ts = int(response.text)
print('Server time: ' + response.text)

# place ask
header = {
	'Accept': 'application/json',
	'Content-Type': 'application/json',
	'X-BTK-APIKEY': API_KEY,
}
data = {
	'sym': 'THB_DOGE',
	'amt': 100, # amount you want to sell
	'rat': 0.27,
	'typ': 'limit',
	'ts': ts,
}
signature = sign(data)
data['sig'] = signature

print('Payload with signature: ' + json_encode(data))
response = requests.post(API_HOST + '/api/market/place-ask', headers=header, data=json_encode(data))


print('Response: ' + response.text)

Server time: 1646841219
Signing payload: {"amt":100,"rat":0.27,"sym":"THB_DOGE","ts":1646841219,"typ":"limit"}
Payload with signature: {"amt":100,"rat":0.27,"sig":"176ca548bb10daeaca1932a6d444aa78e1d4422aa316bb9896e474c0b49c711d","sym":"THB_DOGE","ts":1646841219,"typ":"limit"}
Response: {"error":14}


In [9]:
import hashlib
import hmac
import json
import requests
from pprint import pprint
import pwd
# สร้างไฟล์ pwd.py ขึ้นมา KEY='ddadsfasdfas' , SECRET = b'yasjdkfjaskdf'

# API info
API_HOST = 'https://api.bitkub.com'
API_KEY = 'c55d0fc166f5eff13a648a4323b7f6c5'
API_SECRET = '5ca5732e8e92d972713a2980aeb37bcc'

def json_encode(data):
	return json.dumps(data, separators=(',', ':'), sort_keys=True)

def sign(data):
	j = json_encode(data)
	#print('Signing payload: ' + j)
	h = hmac.new(API_SECRET, msg=j.encode(), digestmod=hashlib.sha256)
	return h.hexdigest()

# check server time
response = requests.get(API_HOST + '/api/servertime')
ts = int(response.text)
#print('Server time: ' + response.text)

# check balances
header = {
	'Accept': 'application/json',
	'Content-Type': 'application/json',
	'X-BTK-APIKEY': API_KEY,
}
data = {
	'ts': ts,
}
signature = sign(data)
data['sig'] = signature

#print('Payload with signature: ' + json_encode(data))
response = requests.post(API_HOST + '/api/market/balances', headers=header, data=json_encode(data))

#print('Balances: ' + response.text)

data = response.json()
data = data['result']
#print(data)
#sym = 'BTC'
sym = ['BTC','DOGE','THB']

for s in sym:
    price = data[s]['available']
    print('{} คงเหลือ: {}'.format(s,price))


TypeError: ignored